In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

from utils.config import domainConfigClass

In [2]:
def generateInvoiceId(date: datetime, code: str, invoiceVersion: int):
    return f'{date.strftime("%d%m%Y")}U{code}{invoiceVersion}'

In [3]:
grn = pd.read_csv("GRN.csv")
pr = pd.read_csv("PR.csv")
grn.drop(columns=["Unnamed: 0"], inplace=True)
pr.drop(columns=["Unnamed: 0"], inplace=True)

grn["Pre Invoice Number"] = grn["Invoice Number"]
pr["Pre Invoice Number"] = pr["Invoice Number"]

In [4]:
swiggyDataset = pd.read_csv("./../../upgrade_mandi/data/cleaned/swiggy/Swiggy.csv")

In [5]:
domain = "Swiggy"
# locationCodeConfig = {
#     "Ayodhya Nagar": "AN",
#     "Byramji": "B",
#     "Dharampeth": "DH",
#     "Mahal": "MH",
#     "Manish Nagar": "MN",
#     "Nandanvan": "NA",
#     "Sai Mandir": "S",
#     "Amravati": "AMD",
# }

locationCodeConfig = {
    location.locationName: location for location in domainConfigClass[domain].locations
}

locationByStoreId = {
    location.storeId: location.locationName
    for location in domainConfigClass[domain].locations
}

In [6]:
def func(row):
    invNo = 1
    try:
        invNo = int(row["Invoice Number"][-1])
    except:
        invNo = 1
    if invNo < 1:
        invNo = 1
    return (
        datetime.strptime(row["Invoice Date"], "%d/%m/%y").strftime("%d%m%Y")
        + "U"
        + locationCodeConfig[locationByStoreId[str(row["Store ID"]).strip()]].code
        + str(invNo)
    )


grn["Invoice No"] = grn.apply(
    func,
    axis=1,
)

pr["Invoice No"] = pr.apply(func, axis=1)

pr.index = pr["Invoice No"]

In [7]:
grn["Location"] = grn.apply(lambda x: locationByStoreId[str(x["Store ID"])], axis=1)
pr["Location"] = pr.apply(lambda x: locationByStoreId[str(x["Store ID"])], axis=1)

In [8]:
# grnPrMerged = grn[
#     [
#         "Invoice Date",
#         "Entity Name",
#         "Invoice No",
#         # "PO Number",
#         "Location",
#         "GRN Amount",
#     ]
# ].copy()
grn["PR"] = grn["Invoice No"].apply(lambda invNo: pr.loc[invNo]["Amount"]).copy()
grn["Invoice Date"] = grn["Invoice Date"].apply(
    lambda date: datetime.strptime(date, "%d/%m/%y").strftime("%d-%m-%Y")
)

In [9]:
grn = grn[grn["PR"].apply(lambda _pr: type(_pr) != pd.Series)]

In [10]:
grn["PR"] = grn["PR"].astype(float).copy()

In [11]:
grn["PO Amount"] = np.nan

grnPrProperColumns = grn[
    [
        "Invoice Date",
        "Invoice No",
        # "PO Number",
        "Location",
        "PO Amount",
        "GRN Amount",
        "PR",
    ]
]

In [12]:
swiggyDataset["Invoice No"] = swiggyDataset.apply(
    lambda row: f'{datetime.strptime(row["Date"], "%m-%d-%Y").strftime("%d%m%Y")}U{locationCodeConfig[row["Location"]].code}{int(row["Invoice Version"])}',
    axis=1,
)
swiggyDataset["Invoice Date"] = swiggyDataset["Date"].apply(
    lambda date: datetime.strptime(date, "%m-%d-%Y").strftime("%d-%m-%Y"),
)

In [13]:
totalAmountDF = swiggyDataset.groupby("Invoice No")["Total Amount"].sum()
groupByDate = swiggyDataset.groupby("Invoice No")

In [14]:
# finalDataFrame["Total Amount"] = (
#     finalDataFrame["Invoice No"]
#     .apply(
#         lambda invNo: (
#             totalAmountDF[invNo] if invNo in totalAmountDF.index else "Missing"
#         )
#     )
#     .copy()
# )

In [15]:
dat = "11-04-2025"
# defaultDF = pd.DataFrame(
#     {
#         "Invoice No": [ for locationCode list(locationCodeConfig.values())],
# 	}
# )
# grnPrOfLocation = finalDataFrame[finalDataFrame["Invoice Date"] == dat].sort_values(
#     "Location"
# )

# tA = groupByDate["Total Amount"].sum(numeric_only=True).reset_index()
# dateDf = groupByDate["Invoice Date"].unique().apply(lambda date: date[0]).reset_index()
# poNoDf = groupByDate["PO No"].unique().apply(lambda poNo: poNo[0]).reset_index()
# locatioDf = groupByDate["Location"].unique().apply(lambda loc: loc[0]).reset_index()

# locatioDf.apply(lambda x: len(x) - 1).all()

# df = pd.DataFrame({"Invoice No": []})
# df = df.merge(locatioDf, how="right", left_on="Invoice No", right_on="Invoice No")
# df = df.merge(dateDf, how="right", left_on="Invoice No", right_on="Invoice No")
# df = df.merge(poNoDf, how="right", left_on="Invoice No", right_on="Invoice No")
# df = df.merge(tA, how="right", left_on="Invoice No", right_on="Invoice No")
# df["Entity Name"] = df["Location"].apply(lambda location: retailerConfig[location])
# df[df["Invoice Date"] == dat].sort_values("Location")
# df["GRN"] = np.nan
# df["PR"] = np.nan

# grnPrOfLocation.merge(
#     df[df["Invoice Date"] == dat], how="right", left_on="Invoice No", right_on="Invoice No"
# )


def listToScaler(row: pd.Series):
    for key in row.keys():
        if type(row[key]) == np.ndarray:
            row[key] = row[key][0]
    return row


grnPrnByInvoice = (
    swiggyDataset.groupby("Invoice No")
    .agg(
        {
            "Total Amount": "sum",
            "Invoice Date": "unique",
            "Location": "unique",
            "PO No": "unique",
        }
    )
    .reset_index()
    .apply(listToScaler, axis=1)
)

grnPrnByInvoice["GRN"] = np.nan
grnPrnByInvoice["PR"] = np.nan

grnPrnByInvoice = grnPrnByInvoice[
    [
        "Invoice Date",
        "Invoice No",
        # "PO No",
        "Location",
        "Total Amount",
        "GRN",
        "PR",
    ]
]

grnPrnByInvoice.columns = grnPrProperColumns.columns = [
    "Invoice Date",
    "Invoice No",
    # "PO Number",
    "Location",
    "PO Amount",
    "GRN",
    "PR",
]

In [16]:
grnPrProperColumns[grnPrProperColumns["Invoice Date"] == dat]

,Invoice Date,Invoice No,Location,PO Amount,GRN,PR
398,11-04-2025,11042025UAN1,Ayodhya Nagar,NaN,7462.0,448.0
399,11-04-2025,11042025UMN1,Manish Nagar,NaN,5705.0,1027.0
400,11-04-2025,11042025UMH1,Mahal,NaN,4144.0,61.0
401,11-04-2025,11042025UB1,Byramji,NaN,65893.0,1882.0
402,11-04-2025,11042025UDH1,Dharampeth,NaN,9055.0,574.0
403,11-04-2025,11042025US1,Sai Mandir,NaN,5408.0,666.0


In [17]:
grnPrnByInvoice[grnPrnByInvoice["Invoice Date"] == dat]

,Invoice Date,Invoice No,Location,PO Amount,GRN,PR
440,11-04-2025,11042025UAN1,Ayodhya Nagar,7910,NaN,NaN
441,11-04-2025,11042025UB1,Byramji,67775,NaN,NaN
442,11-04-2025,11042025UDH1,Dharampeth,9629,NaN,NaN
443,11-04-2025,11042025UMH1,Mahal,4205,NaN,NaN
444,11-04-2025,11042025UMN1,Manish Nagar,6732,NaN,NaN
445,11-04-2025,11042025UNA1,Nandanvan,6338,NaN,NaN
446,11-04-2025,11042025US1,Sai Mandir,6074,NaN,NaN


In [19]:
finalGrnPrDF = pd.merge(
    grnPrnByInvoice,
    grnPrProperColumns,
    how="outer",
    on=[
        "Invoice Date",
        "Invoice No",
        "Location",
    ],
    suffixes=("_T1", "_T2"),
)

finalGrnPrDF["PO Amount"] = finalGrnPrDF["PO Amount_T1"].combine_first(
    finalGrnPrDF["PO Amount_T2"]
)
finalGrnPrDF["GRN"] = finalGrnPrDF["GRN_T1"].combine_first(finalGrnPrDF["GRN_T2"])
finalGrnPrDF["PR"] = finalGrnPrDF["PR_T1"].combine_first(finalGrnPrDF["PR_T2"])

finalGrnPrDF = finalGrnPrDF[
    [
        "Invoice Date",
        "Invoice No",
        "Location",
        "PO Amount",
        "GRN",
        "PR",
    ]
]

In [20]:
finalGrnPrDF["Entity Name"] = finalGrnPrDF["Location"].apply(
    lambda location: locationCodeConfig[location].retailer
)

In [22]:
finalGrnPrDF.to_csv("GRN - PR.csv", index=False)